In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [5]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [6]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 20595 unique tokens.


In [7]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [8]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [10]:
# prepare embedding matrix  (matrix of embeddings of the words in our dataset)
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Dense, Dropout, LSTM
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [0]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')             ## input layer
embedded_sequences = embedding_layer(sequence_input)                            ## embeddings layer
X = LSTM(128, return_sequences=True)(embedded_sequences)
X = Dropout(0.5)(X)
X = LSTM(128, return_sequences=True)(X)
X = Dropout(0.5)(X)
X = keras.layers.core.Reshape((34,128,1))(X)
x1 = Conv2D(2, (2, 100), activation='relu')(X)                 ## conv 1
x1 = GlobalMaxPooling2D()(x1)                                                   ## max 1
x2 = Conv2D(2, (3, 100), activation='relu')(X)                 ## conv 2
x2 = GlobalMaxPooling2D()(x2)                                                   ## max 2
x3 = Conv2D(2, (4, 100), activation='relu')(X)                 ## conv 3
x3 = GlobalMaxPooling2D()(x3)                                                   ## max 3
x  = keras.layers.concatenate([x1, x2, x3])                                     ## concatenate
preds = Dense(2, activation='softmax')(x)                                       ## softmax layer

model = Model(sequence_input, preds)

In [0]:
from keras import backend as K

def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall


      
def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
      

def f1(y_true, y_pred):
    
    Precision = precision(y_true, y_pred)
    Recall = recall(y_true, y_pred)
    f1 = 2*((Precision*Recall)/(Precision+Recall+K.epsilon()))
    
    return f1

In [0]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.3, amsgrad=False)

In [21]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.6669 - acc: 0.6363 - val_loss: 0.6264 - val_acc: 0.7383
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.6234 - acc: 0.7283 - val_loss: 0.5795 - val_acc: 0.7772
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.5859 - acc: 0.7641 - val_loss: 0.5393 - val_acc: 0.8122
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.5558 - acc: 0.7758 - val_loss: 0.5055 - val_acc: 0.8202
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.5284 - acc: 0.7856 - val_loss: 0.4795 - val_acc: 0.8272
Epoch 6/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.5049 - acc: 0.8043 - val_loss: 0.4580 - val_acc: 0.8272
Epoch 7/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.4901 - acc: 0.7973 - val_loss: 0.4410 - val_acc: 0.8312
Epoch 8

In [22]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.3703 - precision: 0.8338 - val_loss: 0.3451 - val_precision: 0.8521
Epoch 2/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3659 - precision: 0.8353 - val_loss: 0.3442 - val_precision: 0.8521
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3653 - precision: 0.8390 - val_loss: 0.3436 - val_precision: 0.8521
Epoch 4/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3649 - precision: 0.8383 - val_loss: 0.3432 - val_precision: 0.8531
Epoch 5/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.3672 - precision: 0.8390 - val_loss: 0.3427 - val_precision: 0.8531
Epoch 6/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3604 - precision: 0.8388 - val_loss: 0.3423 - val_precision: 0.8541
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step -

In [24]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.3480 - recall: 0.8493 - val_loss: 0.3336 - val_recall: 0.8601
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3494 - recall: 0.8435 - val_loss: 0.3333 - val_recall: 0.8611
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3471 - recall: 0.8470 - val_loss: 0.3332 - val_recall: 0.8611
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3447 - recall: 0.8498 - val_loss: 0.3330 - val_recall: 0.8611
Epoch 5/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.3449 - recall: 0.8485 - val_loss: 0.3328 - val_recall: 0.8611
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3523 - recall: 0.8438 - val_loss: 0.3326 - val_recall: 0.8611
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3439 - recall: 0.8433 - val

In [25]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = adam, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.3376 - f1: 0.8515 - val_loss: 0.3287 - val_f1: 0.8611
Epoch 2/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.3359 - f1: 0.8525 - val_loss: 0.3285 - val_f1: 0.8601
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3421 - f1: 0.8423 - val_loss: 0.3284 - val_f1: 0.8601
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3373 - f1: 0.8503 - val_loss: 0.3283 - val_f1: 0.8611
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3371 - f1: 0.8520 - val_loss: 0.3282 - val_f1: 0.8611
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3390 - f1: 0.8523 - val_loss: 0.3281 - val_f1: 0.8611
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3377 - f1: 0.8490 - val_loss: 0.3280 - val_f1: 0.8611
Epoch 8/50
4001/4001 

In [0]:
 model.save_weights('/content/drive/My Drive/Sarcasm Detection/Plots/LSTM_CNN_optimized_Weights.h5')